In [2]:
import sys
sys.path.append('..')

In [3]:
import os
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import defaultdict

In [4]:
from sklearn.metrics import mean_squared_error
from scipy import stats

### Load Data

In [5]:
data_path = '/nethome/kgeorge/workspace/DomainPrediction/Data/fitness_prediction/TEM1'

In [6]:
results_file = os.path.join(data_path, 'results_tem1.csv')
df_results = pd.read_csv(results_file)

In [7]:
df_results

,pos,variant,fitness_raw,fitness_norm,wt_aa,n_mut,seq,fold_id,split_id,pred_esmc_pseudolikelihood,...,pred_esmc_concat_RF,pred_esm2_pseudolikelihood,pred_esm2_wt_marginal,pred_esm2_masked_marginal,pred_esmc_wt_marginal,pred_esmc_masked_marginal,pred_esm2_regfit,pred_esmc_regfit,pred_esmc_confit,pred_esm2_confit
0,58,F58N,1.537240,1.142208,F,1,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,1,2,-345.27032,...,0.709964,-325.914018,-4.302010,-1.232786,0.562500,0.750000,-0.476067,-0.095794,-1.125000,4.958600
1,19,L19W,1.053720,1.094952,L,1,MSIQHFRVALIPFFAAFCWPVFAHPETLVKVKDAEDQLGARVGYIE...,0,2,-352.02673,...,0.529416,-330.761652,-6.557474,-3.308943,-3.375000,-3.265625,-0.605238,-0.187430,-1.562500,6.747321
2,15,A15Y,0.823567,1.072459,A,1,MSIQHFRVALIPFFYAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,0,2,-349.75555,...,0.472685,-327.271534,-6.383676,-2.954151,-3.234375,-2.734375,-0.287633,-0.227720,-0.875000,5.348175
3,182,A182M,0.768011,1.067029,A,1,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,6,0,-347.10516,...,-0.522055,-326.096836,-5.002484,-1.501859,-1.375000,-1.632812,-0.328721,-0.133403,-1.000000,2.113946
4,281,S281F,0.768011,1.067029,S,1,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,9,2,-350.15952,...,0.275112,-331.592502,-6.108358,-2.640135,-1.625000,-2.078125,-0.252322,-0.182261,-1.062500,3.514660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5193,130,N130W,-10.232000,-0.008031,N,1,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,4,2,-363.09357,...,-9.665906,-340.835044,-15.097270,-13.994238,-15.437033,-15.874519,-9.808790,-8.350313,-16.373184,-22.297573
5194,241,R241P,-10.232000,-0.008031,R,1,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,8,1,-364.08163,...,-7.171978,-337.134499,-12.711793,-11.034103,-14.687071,-15.248886,-5.181339,-8.563768,-10.242188,-17.278234
5195,255,R255P,-10.232000,-0.008031,R,1,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,8,1,-356.27032,...,-4.108840,-335.036729,-12.087317,-7.394194,-6.431641,-7.173828,-0.404985,-6.293775,-9.240234,-15.969477
5196,227,W227P,-10.232000,-0.008031,W,1,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,7,2,-356.40576,...,-8.686791,-336.644174,-12.189831,-10.138597,-9.887451,-8.170898,-9.742272,-9.974316,-16.992676,-24.078823


In [8]:
df_results.columns.str.contains('pred').sum()

23

### Useful Functions

In [9]:
def get_spearmanr_bootstrap(a, b, n=1000, ci = 95):
    assert len(a) == len(b)
    corr = []
    p_values = []
    np.random.seed(0)
    for _ in range(n):
        indices = np.random.choice(len(a), size=len(a), replace=True)
        res = stats.spearmanr(a[indices], b[indices])
        
        if not np.isnan(res.statistic):
            corr.append(res.statistic)
            p_values.append(res.pvalue)
    
    if len(corr):
        ci_lower, ci_upper = np.percentile(corr, [100-ci, ci]) 
        mean_corr = np.mean(corr)
        p_value = np.mean(np.array(corr) < 0)

        return round(mean_corr, 2), round(ci_lower, 2), round(ci_upper, 2), round(p_value, 2), corr, p_values
    else:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

In [10]:
def get_split_mask(df, omit_zero=False):
    if omit_zero:
        train_mask = (df['split_id'] == 2) & (df['fitness_raw'] != 0)
    else:
        train_mask = (df['split_id'] == 2)

    val_mask = df['split_id'] == 1
    test_mask = df['split_id'] == 0

    return train_mask, val_mask, test_mask

In [11]:
name_dict = {
        'pred_OHE_ridge': 'OHE Linear',
        'pred_OHE_RF': 'OHE RF',
        'pred_OHE_MLP': 'OHE MLP',

        'pred_esm2_mean_ridge': 'ESM2 mean-embeddings Linear',
        'pred_esm2_mean_RF': 'ESM2 mean-embeddings RF',
        'pred_esm2_mean_MLP': 'ESM2 mean-embeddings MLP',
        'pred_esm2_concat_ridge': 'ESM2 concat-embeddings Linear',
        'pred_esm2_concat_RF': 'ESM2 concat-embeddings RF',

        'pred_esmc_mean_ridge': 'ESMC mean-embeddings Linear',
        'pred_esmc_mean_RF': 'ESMC mean-embeddings RF',
        'pred_esmc_mean_MLP': 'ESMC mean-embeddings MLP',
        'pred_esmc_concat_ridge': 'ESMC concat-embeddings Linear',
        'pred_esmc_concat_RF': 'ESMC concat-embeddings RF',

        'pred_esm2_wt_marginal': 'ESM2 wildtype marginal',
        'pred_esm2_masked_marginal': 'ESM2 masked marginal',
        'pred_esm2_pseudolikelihood': 'ESM2 pseudolikelihood',

        'pred_esmc_wt_marginal': 'ESMC wildtype marginal',
        'pred_esmc_masked_marginal': 'ESMC masked marginal',
        'pred_esmc_pseudolikelihood': 'ESMC pseudolikelihood',

        'pred_esm2_regfit': 'ESM2 regression finetuning',
        'pred_esmc_regfit': 'ESMC regression finetuning',

        'pred_esm2_confit': 'ESM2 contrastive finetuning',
        'pred_esmc_confit': 'ESMC contrastive finetuning',
        
    }

### All Plots

In [ ]:
def get_plot_check(df, label, fit_label='fitness_log'):
    train_mask, val_mask, test_mask = get_split_mask(df)
    assert fit_label in df.columns

    fig, ax = plt.subplots(1,4, figsize=(13,3), layout='constrained')

    omit_mask = df['fitness_raw'] > -100

    ax[0].plot(df.loc[train_mask, fit_label], df.loc[train_mask, label], '.', alpha=0.9)
    train_corr, train_ci_lower, train_ci_upper, p_value, *_  = get_spearmanr_bootstrap(df.loc[train_mask, fit_label].to_numpy(), df.loc[train_mask, label].to_numpy())
    ax[0].set_title(f'Train\n corr {train_corr} CI ({train_ci_lower}, {train_ci_upper}) P value {p_value}')

    ax[1].plot(df.loc[train_mask, fit_label], df.loc[train_mask, label], '.', alpha=0.9)
    ax[1].plot(df.loc[train_mask & omit_mask, fit_label], df.loc[train_mask & omit_mask, label], '.', alpha=0.9)
    train_corr, train_ci_lower, train_ci_upper, p_value, *_  = get_spearmanr_bootstrap(df.loc[train_mask & omit_mask, fit_label].to_numpy(), df.loc[train_mask & omit_mask, label].to_numpy())
    ax[1].set_title(f'Train Omit\n corr {train_corr} CI ({train_ci_lower}, {train_ci_upper}) P value {p_value}')

    ax[2].plot(df.loc[test_mask, fit_label], df.loc[test_mask, label], '.', alpha=0.9)
    test_corr, test_ci_lower, test_ci_upper, p_value, *_ = get_spearmanr_bootstrap(df.loc[test_mask, fit_label].to_numpy(), df.loc[test_mask, label].to_numpy())
    ax[2].set_title(f'Test\n corr {test_corr} CI ({test_ci_lower}, {test_ci_upper}) P value {p_value}')

    ax[3].plot(df.loc[test_mask, fit_label], df.loc[test_mask, label], '.', alpha=0.9)
    ax[3].plot(df.loc[test_mask & omit_mask, fit_label], df.loc[test_mask & omit_mask, label], '.', alpha=0.9)
    test_corr, test_ci_lower, test_ci_upper, p_value, *_ = get_spearmanr_bootstrap(df.loc[test_mask & omit_mask, fit_label].to_numpy(), df.loc[test_mask & omit_mask, label].to_numpy())
    ax[3].set_title(f'Test\n corr {test_corr} CI ({test_ci_lower}, {test_ci_upper}) P value {p_value}')
    
    for i in range(4):
        ax[i].set_xlabel('True')
        ax[i].set_ylabel('Pred')

    plt.show()

In [ ]:
for label in df_results.columns[df_results.columns.str.contains('pred')]:
    print(label)
    
    get_plot_check(df_results, label, fit_label='fitness_raw')

### Tables

In [ ]:
def get_table(df, fit_label='fitness_log', omit=True, omit_label=None):
    table = {}
    train_mask, val_mask, test_mask = get_split_mask(df)
    assert fit_label in df.columns
    for label in df.columns[df.columns.str.contains('pred')]:
        if omit:
            assert omit_label is not None and omit_label in df.columns
            omit_mask = df[omit_label] > 0
            train_corr, train_ci_lower, train_ci_upper, train_p_value, *_  = get_spearmanr_bootstrap(df.loc[train_mask & omit_mask, fit_label].to_numpy(), df.loc[train_mask & omit_mask, label].to_numpy())
            test_corr, test_ci_lower, test_ci_upper, test_p_value, *_ = get_spearmanr_bootstrap(df.loc[test_mask & omit_mask, fit_label].to_numpy(), df.loc[test_mask & omit_mask, label].to_numpy())
        else:
            train_corr, train_ci_lower, train_ci_upper, train_p_value, *_  = get_spearmanr_bootstrap(df.loc[train_mask, fit_label].to_numpy(), df.loc[train_mask, label].to_numpy())
            test_corr, test_ci_lower, test_ci_upper, test_p_value, *_ = get_spearmanr_bootstrap(df.loc[test_mask, fit_label].to_numpy(), df.loc[test_mask, label].to_numpy())
        
        table[label] = [train_corr, (train_ci_lower, train_ci_upper), train_p_value, test_corr, (test_ci_lower, test_ci_upper), test_p_value]

    df_table = pd.DataFrame(table).T
    df_table.columns = ['train', 'train CI', 'train p-val', 'test', 'test CI', 'test p-val']

    return df_table

In [ ]:
res_table = get_table(df_results, omit=False, fit_label='fitness_raw')

In [ ]:
res_table

In [ ]:
res_table.sort_values(by='test', ascending=False).index

In [ ]:
res_table[res_table['test p-val'] < 0.05]

In [ ]:
res_table[res_table['test CI'].apply(lambda x: x[0] > 0)]

In [ ]:
temp_table = {}
for i, row in res_table.iterrows():
    temp_table[row.name] = [f"{row['test']} {row['test CI']}"]
temp_table = pd.DataFrame(temp_table).T

In [ ]:
temp_table

### Panel

In [ ]:
def get_panel(df, label, fit_label='fitness_log', omit=True, omit_label=None):
    train_mask, val_mask, test_mask = get_split_mask(df)
    assert fit_label in df.columns

    fig, ax = plt.subplots(1,2, figsize=(7,3), layout='constrained')

    if omit:
        assert omit_label is not None and omit_label in df.columns
        
        omit_mask = df[omit_label] > 0

        ax[0].plot(df.loc[train_mask & omit_mask, fit_label], df.loc[train_mask & omit_mask, label], '.', alpha=0.9)
        ax[1].plot(df.loc[test_mask & omit_mask,fit_label], df.loc[test_mask & omit_mask, label], '.', alpha=0.9)

        train_corr, train_ci_lower, train_ci_upper, *_  = get_spearmanr_bootstrap(df.loc[train_mask & omit_mask, fit_label].to_numpy(), df.loc[train_mask & omit_mask, label].to_numpy())
        test_corr, test_ci_lower, test_ci_upper, *_ = get_spearmanr_bootstrap(df.loc[test_mask & omit_mask, fit_label].to_numpy(), df.loc[test_mask & omit_mask, label].to_numpy())

        ax[0].set_title(f'Train\nspearman correlation {train_corr} ({train_ci_lower}, {train_ci_upper})', size=10)
        ax[1].set_title(f'Test\nspearman correlation {test_corr} ({test_ci_lower}, {test_ci_upper})', size=10)
    else:
        ax[0].plot(df.loc[train_mask, fit_label], df.loc[train_mask, label], '.', alpha=0.9)
        ax[1].plot(df.loc[test_mask, fit_label], df.loc[test_mask, label], '.', alpha=0.9)

        train_corr, train_ci_lower, train_ci_upper, *_  = get_spearmanr_bootstrap(df.loc[train_mask, fit_label].to_numpy(), df.loc[train_mask, label].to_numpy())
        test_corr, test_ci_lower, test_ci_upper, *_ = get_spearmanr_bootstrap(df.loc[test_mask, fit_label].to_numpy(), df.loc[test_mask, label].to_numpy())

        ax[0].set_title(f'Train\nspearman correlation {train_corr} ({train_ci_lower}, {train_ci_upper})', size=10)
        ax[1].set_title(f'Test\nspearman correlation {test_corr} ({test_ci_lower}, {test_ci_upper})', size=10)

    name_dict = {
        'pred_OHE_ridge': 'OHE Linear',
        'pred_OHE_RF': 'OHE RF',
        'pred_OHE_MLP': 'OHE MLP',

        'pred_esm2_mean_ridge': 'ESM2 mean-embeddings Linear',
        'pred_esm2_mean_RF': 'ESM2 mean-embeddings RF',
        'pred_esm2_mean_MLP': 'ESM2 mean-embeddings MLP',
        'pred_esm2_concat_ridge': 'ESM2 concat-embeddings Linear',
        'pred_esm2_concat_RF': 'ESM2 concat-embeddings RF',

        'pred_esmc_mean_ridge': 'ESMC mean-embeddings Linear',
        'pred_esmc_mean_RF': 'ESMC mean-embeddings RF',
        'pred_esmc_mean_MLP': 'ESMC mean-embeddings MLP',
        'pred_esmc_concat_ridge': 'ESMC concat-embeddings Linear',
        'pred_esmc_concat_RF': 'ESMC concat-embeddings RF',

        'pred_esm2_wt_marginal': 'ESM2 wildtype marginal',
        'pred_esm2_masked_marginal': 'ESM2 masked marginal',
        'pred_esm2_pseudolikelihood': 'ESM2 pseudolikelihood',

        'pred_esmc_wt_marginal': 'ESMC wildtype marginal',
        'pred_esmc_masked_marginal': 'ESMC masked marginal',
        'pred_esmc_pseudolikelihood': 'ESMC pseudolikelihood',
    }
    
    fig.suptitle(name_dict[label])
    
    for i in range(2):
        ax[i].set_xlabel('True')
        ax[i].set_ylabel('Pred')

    plt.show()

In [ ]:
get_panel(df_results, 'pred_esm2_masked_marginal', omit=False, fit_label='fitness_raw')

### Comparison

In [ ]:
def get_diff(y, a, b, n=1000, ci = 95):

    assert len(y) == len(a) == len(b)
    
    corr_a_list = []
    corr_b_list = []
    diff_corr = []
    np.random.seed(0)
    
    for _ in range(n):
        indices = np.random.choice(len(a), size=len(a), replace=True)
        corr_a = stats.spearmanr(y[indices], a[indices]).statistic
        corr_b = stats.spearmanr(y[indices], b[indices]).statistic
        
        assert not np.isnan(corr_a)
        assert not np.isnan(corr_b)

        corr_a_list.append(corr_a)
        corr_b_list.append(corr_b)

        diff_corr.append(corr_a - corr_b)

    ci_lower, ci_upper = np.percentile(diff_corr, [5, 95]) 
    p_value = np.mean(np.array(diff_corr) < 0)

    # plt.hist(corr_b_list)
    # plt.hist(corr_a_list)
    
    return p_value, ci_lower, ci_upper

In [ ]:
train_mask, val_mask, test_mask = get_split_mask(df_results)

get_diff(df_results.loc[test_mask, 'fitness_raw'].to_numpy(), df_results.loc[test_mask, 'pred_esmc_confit'].to_numpy(), df_results.loc[test_mask, 'pred_esm2_confit'].to_numpy())

In [ ]:
order_list = ['pred_esmc_confit', 'pred_esm2_confit', 'pred_esmc_concat_ridge',
       'pred_esmc_masked_marginal', 'pred_esm2_masked_marginal',
       'pred_esmc_concat_RF', 'pred_esmc_wt_marginal', 'pred_esm2_wt_marginal',
       'pred_esmc_mean_MLP', 'pred_esmc_pseudolikelihood',
       'pred_esm2_pseudolikelihood', 'pred_esmc_mean_RF', 'pred_esmc_regfit',
       'pred_esmc_mean_ridge', 'pred_esm2_concat_RF', 'pred_esm2_mean_RF',
       'pred_esm2_mean_MLP', 'pred_esm2_regfit', 'pred_esm2_mean_ridge',
       'pred_esm2_concat_ridge', 'pred_OHE_MLP']

for i in range(len(order_list)):
    print(f'{order_list[i]}:-')
    for j in range(i+1, len(order_list)):

        p, *_ = get_diff(df_results.loc[test_mask, 'fitness_raw'].to_numpy(), 
                         df_results.loc[test_mask, order_list[i]].to_numpy(), 
                         df_results.loc[test_mask, order_list[j]].to_numpy())
        
        if p >= 0.05:
            print(order_list[i], order_list[j], p)

    print(' ')

In [ ]:
order_list = ['pred_esmc_confit', 'pred_esm2_confit', 'pred_esmc_concat_ridge', 'pred_esmc_masked_marginal', 'pred_esm2_masked_marginal']
significance_matrix = np.zeros((len(order_list), len(order_list)))
for i in range(len(order_list)):
    for j in range(len(order_list)):

        p, *_ = get_diff(df_results.loc[test_mask, 'fitness_raw'].to_numpy(), 
                         df_results.loc[test_mask, order_list[i]].to_numpy(), 
                         df_results.loc[test_mask, order_list[j]].to_numpy())
        
        significance_matrix[i,j] = p

In [ ]:
significance_matrix

In [ ]:
plt.imshow(significance_matrix < 0.05)

### ESMC vs ESM2

In [ ]:
def get_diff(y, a, b, n=1000, ci = 95):

    assert len(y) == len(a) == len(b)
    
    corr_a_list = []
    corr_b_list = []
    diff_corr = []
    np.random.seed(0)
    
    for _ in range(n):
        indices = np.random.choice(len(a), size=len(a), replace=True)
        corr_a = stats.spearmanr(y[indices], a[indices]).statistic
        corr_b = stats.spearmanr(y[indices], b[indices]).statistic
        
        assert not np.isnan(corr_a)
        assert not np.isnan(corr_b)

        corr_a_list.append(corr_a)
        corr_b_list.append(corr_b)

        diff_corr.append(corr_a - corr_b)

    ci_lower, ci_upper = np.percentile(diff_corr, [5, 95]) 
    p_value = np.mean(np.array(diff_corr) < 0)

    # plt.hist(corr_b_list)
    # plt.hist(corr_a_list)
    
    return p_value, corr_a_list, corr_b_list

In [ ]:
esmc_indicater = ['pred_esmc_mean_ridge', 'pred_esmc_mean_RF', 'pred_esmc_mean_MLP', 'pred_esmc_concat_ridge', 'pred_esmc_concat_RF',
                  'pred_esmc_wt_marginal', 'pred_esmc_masked_marginal', 'pred_esmc_pseudolikelihood', 'pred_esmc_regfit', 
                  'pred_esmc_confit']
esm2_indicater = ['pred_esm2_mean_ridge', 'pred_esm2_mean_RF', 'pred_esm2_mean_MLP', 'pred_esm2_concat_ridge', 'pred_esm2_concat_RF',
                  'pred_esm2_wt_marginal', 'pred_esm2_masked_marginal', 'pred_esm2_pseudolikelihood', 'pred_esm2_regfit', 
                  'pred_esm2_confit']

In [ ]:
train_mask, val_mask, test_mask = get_split_mask(df_results)

fig, ax = plt.subplots(figsize=(10, 4), layout='constrained')
ax.set_xticks(range(len(esmc_indicater)))
ax.set_xticklabels([f"{name_dict.get(x, x).replace('ESMC ', '')}" for x in esmc_indicater], rotation=45, ha='right')

for a, b in zip(esmc_indicater, esm2_indicater):
    p, dist_a, dist_b = get_diff(df_results.loc[test_mask, 'fitness_raw'].to_numpy(), 
                         df_results.loc[test_mask, a].to_numpy(), 
                         df_results.loc[test_mask, b].to_numpy())
    
    # Calculate mean and confidence intervals
    mean_a, mean_b = np.mean(dist_a), np.mean(dist_b)
    ci_a = np.percentile(dist_a, [5, 95])
    ci_b = np.percentile(dist_b, [5, 95])

    x_pos = len(ax.patches) // 2
    if len(ax.patches) == 0:
        ax.bar(x_pos - 0.2, mean_a, yerr=[[mean_a - ci_a[0]], [ci_a[1] - mean_a]], capsize=5, color='lightseagreen', width=0.4, label='ESMC')
        ax.bar(x_pos + 0.2, mean_b, yerr=[[mean_b - ci_b[0]], [ci_b[1] - mean_b]], capsize=5, color='burlywood', width=0.4, label='ESM2')
    else:
        ax.bar(x_pos - 0.2, mean_a, yerr=[[mean_a - ci_a[0]], [ci_a[1] - mean_a]], capsize=5, color='lightseagreen', width=0.4)
        ax.bar(x_pos + 0.2, mean_b, yerr=[[mean_b - ci_b[0]], [ci_b[1] - mean_b]], capsize=5, color='burlywood', width=0.4)

    # Add significance marker
    if p < 0.05:
        ax.text(x_pos, max(mean_a, mean_b) - 0.01, '**', ha='center', va='bottom', fontsize=12, color='red')

ax.set_ylabel('Spearman Correlation')
ax.legend()
plt.show()    


### concat vs mean

In [12]:
def get_diff(y, a, b, n=1000, ci = 95):

    assert len(y) == len(a) == len(b)
    
    corr_a_list = []
    corr_b_list = []
    diff_corr = []
    np.random.seed(0)
    
    for _ in range(n):
        indices = np.random.choice(len(a), size=len(a), replace=True)
        corr_a = stats.spearmanr(y[indices], a[indices]).statistic
        corr_b = stats.spearmanr(y[indices], b[indices]).statistic
        
        assert not np.isnan(corr_a)
        assert not np.isnan(corr_b)

        corr_a_list.append(corr_a)
        corr_b_list.append(corr_b)

        diff_corr.append(corr_a - corr_b)

    ci_lower, ci_upper = np.percentile(diff_corr, [5, 95]) 
    p_value = np.mean(np.array(diff_corr) < 0)

    # plt.hist(corr_b_list)
    # plt.hist(corr_a_list)
    
    return p_value, ci_lower, ci_upper

In [13]:
train_mask, val_mask, test_mask = get_split_mask(df_results)

In [14]:
concat_list = ['pred_esmc_concat_ridge', 'pred_esmc_concat_RF']
mean_list = ['pred_esmc_mean_ridge', 'pred_esmc_mean_RF', 'pred_esmc_mean_MLP']
significance_matrix = np.zeros((len(concat_list), len(mean_list)))
for a in concat_list:
    for b in mean_list:

        p, *_ = get_diff(df_results.loc[test_mask, 'fitness_raw'].to_numpy(), 
                         df_results.loc[test_mask, a].to_numpy(), 
                         df_results.loc[test_mask, b].to_numpy())

        significance_matrix[concat_list.index(a), mean_list.index(b)] = p

significance_matrix

array([[0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.22]])